# Использование RAG с помощью Azure AI Agent Service и Semantic Kernel

Этот пример кода демонстрирует, как создать и управлять агентом Azure AI для генерации с дополнением извлеченной информации (RAG) с использованием `Azure AI Agent Service` и `Semantic Kernel`. Агент обрабатывает запросы пользователей на основе извлеченного контекста и предоставляет точные ответы соответственно.


Исправление версии SQLite  
Если вы столкнулись с ошибкой:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Раскомментируйте этот блок кода в начале вашего ноутбука:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Импорт пакетов
Следующий код импортирует необходимые пакеты:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Генерация с использованием поиска с помощью Semantic Kernel и Azure AI Agent Service

Этот пример демонстрирует, как использовать **Azure AI Agent Service** для выполнения **генерации с использованием поиска (RAG)**, объединяя языковую модель с контекстом, специфичным для области, из загруженного документа.

### Как это работает

1. **Загрузка документа**: В агент-сервис загружается файл в формате markdown (document.md), содержащий информацию (страховая политика Contoso для путешествий).

2. **Создание векторного хранилища**: Документ индексируется в векторное хранилище, чтобы обеспечить семантический поиск по его содержимому.

3. **Настройка агента**: Агент создается с использованием модели `gpt-4o` с следующими строгими инструкциями:
   - Отвечать на вопросы только на основе извлеченного содержимого из документа.
   - Отказываться отвечать, если вопрос выходит за рамки доступной информации.

4. **Интеграция инструмента поиска файлов**: `FileSearchTool` регистрируется в агенте, позволяя модели искать и извлекать релевантные фрагменты из индексированного документа во время выполнения запроса.

5. **Взаимодействие с пользователем**: Пользователи могут задавать вопросы. Если в документе найдена релевантная информация, агент генерирует обоснованный ответ.  
   Если информации недостаточно, агент явно сообщает, что документ не содержит достаточных данных.


### Основная функция



Убедитесь, что сначала выполнена команда `az login` с использованием Azure CLI, чтобы обеспечить правильный контекст аутентификации при использовании `DefaultAzureCredential`. Служба Azure AI Agent не использует ключи API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Пожалуйста, предоставьте текст файла Markdown, который нужно перевести.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
